In [34]:
import pandas as pd
import plotly.graph_objs as go 
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from tqdm import tqdm 
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Import des données

In [35]:
apple = pd.read_csv("../data/AAPL.csv")
amazon = pd.read_csv("../data/AMZN.csv")
berkshire = pd.read_csv("../data/BRK-B.csv")
google = pd.read_csv("../data/GOOG.csv")
meta = pd.read_csv("../data/META.csv")
microsoft = pd.read_csv("../data/MSFT.csv")
nvidia = pd.read_csv("../data/NVDA.csv")
tesla = pd.read_csv("../data/TSLA.csv")
unitedhealth = pd.read_csv("../data/UNH.csv")
exxon = pd.read_csv("../data/XOM.csv")

In [36]:
titres = {"Apple" : apple, 
          "Amazon" : amazon,
          "Berkshire" : berkshire,
          "Google" : google,
          "Meta" : meta,
          "Microsoft" : microsoft,
          "Nvidia" : nvidia,
          "Tesla" : tesla,
          "UnitedHealth" : unitedhealth,
          "Exxon" : exxon}

In [37]:
apple.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

## test de FE

# Calcul des prédictions

In [38]:
def stock_forecast(stock):

    stock['Date'] = pd.to_datetime(stock['Date'])
    stock['Adj Close'] = stock['Adj Close'].pct_change()
    stock['High'] = stock['High'].pct_change()
    stock['Low'] = stock['Low'].pct_change()
    stock['Volume'] = stock['Volume'].pct_change()
    stock['dailyChange'] = (stock['Adj Close'] - stock['Open']) / stock['Open']
    # stock['5EMA'] = (stock['Adj Close'].ewm(span=5,adjust=True,ignore_na=True).mean())
    # stock['10EMA'] = (stock['Adj Close'].ewm(span=10,adjust=True,ignore_na=True).mean())
    stock['priceDirection'] = (stock['Adj Close'].shift(-1) - stock['Adj Close'])
    rsi_period = 10
    chg = stock['Adj Close'].diff(1)
    gain = chg.mask(chg<0,0)
    loss = chg.mask(chg>0,0)
    avg_gain = gain.ewm(com = rsi_period - 1, min_periods = rsi_period).mean()
    avg_loss = loss.ewm(com = rsi_period - 1, min_periods = rsi_period).mean()
    rs = abs(avg_gain/avg_loss)
    stock['rsi'] = 100-(100/(1+ rs))
    stock['Std_dev']=  stock['Adj Close'].rolling(10).std()
    # stock['ROC'] = ((stock['Adj Close'] - stock['Adj Close'].shift(5)) / (stock['Adj Close'].shift(10)))*100
    stock['Williams%R'] = (stock['High'].max() - stock['Adj Close'])/(stock['High'] - stock['Low'].min()) * -100
    stock = stock.replace([np.inf, -np.inf], np.nan)
    stock.dropna(inplace=True)
    stock['signal'] = 0.0
    stock['signal'] = np.where((stock.loc[:,'priceDirection'] > 0), 1.0, 0.0)
    stock.drop(columns =['Open','priceDirection' ], axis=1, inplace=True)
    stock['Volatility_HighLow_10days'] = stock['High'].rolling(window=10).std() / stock['Low'].rolling(window=10).mean()

    stock['Return'] = stock['Close'].pct_change()
    stock = stock.dropna()
    stock = stock.drop(columns=['Adj Close'])

    stock['Target'] = stock['Return'].shift(-10)
    stock = stock.dropna()
    
    window_size = 90
    step_size = 7  
    prediction_days = 7
    
    prediction_dates = []
    predictions = []
  
    features_to_drop = ['Date', 'Target', 'Volume']
    for start in tqdm(range(0, len(stock) - window_size - prediction_days, step_size)):
        end = start + window_size
        train = stock.iloc[start:end]
        test = stock.iloc[end:end + prediction_days]
    
        model = XGBRegressor(n_estimators=400, max_depth=30, learning_rate=0.01, verbosity=0)
        # grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')

        model.fit(train.drop(features_to_drop, axis=1), train['Target'])
        # best_model = grid_search.best_estimator_
        # model.fit(train.drop(['Date', 'Target', 'Volume'], axis=1), train['Target'])
    
        pred = model.predict(test.drop(['Date', 'Target', 'Volume'], axis=1))
        predictions.append(pred)
    
        prediction_date = stock.iloc[end + prediction_days - 1]['Date']
        prediction_dates.append(prediction_date)

    predictions = [value[6] for value in predictions]


    df_predictions = pd.DataFrame({
        'Date': prediction_dates,
        'Prediction': predictions
    })
    
    merged_df = stock.merge(df_predictions, on='Date', how='inner')
    
    merged_df = merged_df.drop(columns = ["Target"])
    
    return merged_df

In [39]:
predictions_dic = {}

In [40]:
for key in titres :
    predictions_dic[key] = stock_forecast(titres[key])

  0%|          | 0/378 [00:00<?, ?it/s]

 10%|▉         | 36/378 [00:30<06:58,  1.22s/it]

In [ ]:
dfs_to_concat = []

# Boucle sur chaque DataFrame dans le dictionnaire
for name, df in predictions_dic.items():
    # Assurez-vous que la colonne 'Date' est de type datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Ajout du DataFrame à la liste
    dfs_to_concat.append(df[['Date', 'Prediction', "Return"]])

# Concaténation des DataFrames
portfolio_predictions = pd.concat(dfs_to_concat)

# Calcul de la prédiction moyenne pour chaque date
portfolio_avg_predictions = portfolio_predictions.groupby('Date').mean().reset_index()

# Affichage des premières lignes pour vérifier
print(portfolio_avg_predictions.head())

        Date  Prediction    Return
0 2013-06-19   -0.003176 -0.003283
1 2013-06-20    0.001129 -0.021429
2 2013-06-21   -0.007204  0.002963
3 2013-06-25    0.005687 -0.001483
4 2013-06-28   -0.009466  0.000574


In [ ]:
fig = go.Figure()

# Ajout de la série financière
fig.add_trace(go.Scatter(x = portfolio_avg_predictions.Date, 
                         y = portfolio_avg_predictions['Prediction'],
                         mode = 'lines',
                         name ='Return_predict', 
                         fill = "tozeroy"))

fig.add_trace(go.Scatter(x = portfolio_avg_predictions.Date, 
                         y = portfolio_avg_predictions["Return"],
                         mode = 'lines',
                         name ='Return_reel', 
                         fill = "tozeroy"))
# Mise à jour des titres
fig.update_layout(title="Évolution de la valeur de ptf",
                  xaxis_title='Date',
                  yaxis_title='Valeur')

# Affichage du graphique
fig.show()